# DEPENDÊNCIAS

In [ ]:
! pip install spacy
! pip install nltk
! pip install pandas
! pip install pdfplumber
! pip install chromadb
! pip install sentence-transformers
! pip install openpyxl
! pip install python-docx
! pip install langchain-text-splitters
! python -m spacy download pt_core_news_sm

# IMPORTS

In [1]:
import pdfplumber
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from nltk.stem.snowball import SnowballStemmer
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb

d:\github\Walmir-p2-preparation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pdf_path = "./documents/sample_vangogh.pdf"

# Carrega PDF

In [11]:
# Utiliza a bibliotéca pdfplumber para extrair o texto do PDF, o resultado é guardado na variável raw_text
def load_pdf(filepath):
    with pdfplumber.open(filepath) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text



raw_text = load_pdf(pdf_path)
print("Prova de Conceito:\n\n" + raw_text[:500])



Prova de Conceito:

Prólogo
Um coração fanático
Theo imaginou o pior. A mensagem dizia apenas que Vincent tinha “se ferido”.
Enquanto corria até a estação para pegar o próximo trem até Auvers, seus pensa-
mentos vagavam de trás para diante. Da última vez que havia recebido uma mensa-
gem parecida, era um telegrama de Paul Gauguin informando que Vincent estava
“gravemente enfermo”. Theo chegara à cidade meridional de Arles e encontrara o
irmão na enfermaria de um hospital, com a cabeça enfaixada e perdido em desvari


# PLN

## Lowercasing

In [25]:
# Transforma todo o texto em letras minúsculas
text_lower = raw_text.lower()




print("Texto em lowercase (prévia):\n\n", text_lower[:800])

Texto em lowercase (prévia):

 prólogo
um coração fanático
theo imaginou o pior. a mensagem dizia apenas que vincent tinha “se ferido”.
enquanto corria até a estação para pegar o próximo trem até auvers, seus pensa-
mentos vagavam de trás para diante. da última vez que havia recebido uma mensa-
gem parecida, era um telegrama de paul gauguin informando que vincent estava
“gravemente enfermo”. theo chegara à cidade meridional de arles e encontrara o
irmão na enfermaria de um hospital, com a cabeça enfaixada e perdido em desvarios.
dessa vez, o que encontraria ao final da viagem?
em momentos assim — e eram muitos —, theo retornava às lembranças do
vincent que tinha conhecido no passado: um irmão mais velho ardoroso e irrequie-
to, mas também cheio de brincadeiras animadas, uma enorme afinidade e uma infi-
nita capacidade d


## Remover caracteres especiais

In [30]:
def remove_special_chars(text):
    # Normaliza acentos (NFKD)
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ASCII", "ignore").decode("utf-8")

    # Trata hifens seguidos por quebra de linha como quebra de palavra
    text = re.sub(r"-\n", "", text);
    # Remove caracteres que não sejam letras, numeros ou espaços
    text = re.sub(r"[^a-z0-9.,\s]", " ", text)

    # Remove múltiplos espaços
    text = re.sub(r"\s+", " ", text)

    return text.strip()




text_clean = remove_special_chars(text_lower)

print("Texto sem caracteres especiais:\n\n", text_clean[:800])


Texto sem caracteres especiais:

 prologo um coracao fanatico theo imaginou o pior. a mensagem dizia apenas que vincent tinha se ferido. enquanto corria ate a estacao para pegar o proximo trem ate auvers, seus pensamentos vagavam de tras para diante. da ultima vez que havia recebido uma mensagem parecida, era um telegrama de paul gauguin informando que vincent estava gravemente enfermo. theo chegara a cidade meridional de arles e encontrara o irmao na enfermaria de um hospital, com a cabeca enfaixada e perdido em desvarios. dessa vez, o que encontraria ao final da viagem em momentos assim e eram muitos , theo retornava as lembrancas do vincent que tinha conhecido no passado um irmao mais velho ardoroso e irrequieto, mas tambem cheio de brincadeiras animadas, uma enorme afinidade e uma infinita capacidade de admiracao. em s


## Remover Stopwords

In [31]:
# A tokenização sera feita com spaCy pois funciona melhor com português, em comparação com nltk.
# Já as stopwords foram resgatadas com o NLTK, pois a lista de stopwords em português do spaCy não foi tão boa quanto a do NLTK nos testes que fiz (removendo palavras importantes, como 'não').

# Carregar modelo de português
nlp = spacy.load("pt_core_news_sm")

# Baixar recursos necessários do NLTK para stopwords
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
stopwords_pt = set(stopwords.words("portuguese"))



# tokeniza utilizando spacy, e então remove stopwords
token = nlp(text_clean)
tokens_no_stop = [t for t in token if t.text not in stopwords_pt]

print("\nTokens sem stopwords (primeiros 30):\n\n", tokens_no_stop[:30])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GamingPc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\GamingPc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GamingPc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Tokens sem stopwords (primeiros 30):

 [prologo, coracao, fanatico, theo, imaginou, pior, ., mensagem, dizia, apenas, vincent, ferido, ., enquanto, corria, ate, estacao, pegar, proximo, trem, ate, auvers, ,, pensamentos, vagavam, tras, diante, ., ultima, vez]


## Lemmatização

In [105]:
# lemmatização utilizando spaCy, pegando o lemma_ dos tokens.
# utilizando lemmatização, pois ela preserva melhor a semântica das palavras em comparação com stemming.
# stemming não funciona tão bem quando é necessário manter a semântica das palavras.
def lemmatize_text(tokens):
    return [token.lemma_ for token in tokens]




lemmas = " ".join(lemmatize_text(tokens_no_stop))

print("Lemmas (primeiros 30):\n\n", lemmas[:300])


Lemmas (primeiros 30):

 prologo coracao fanatico theo imaginar mau . mensagem dizer apenas vincent ferir . enquanto correr ate estacao pegar proximo tr ate auver , pensamento vagar tra diante . ultimo vez haver receber mensagem parecido , telegrama paul gauguin informar vincent gravemente enfermo . theo chegar cidade merid


## Stemming

In [39]:
# stemmatização, é melhor para quando a semântica não é tão importante, e se prioriza a performance.
# exemplo: search engines, ou detecção de spam.

# Criar stemmer para português
stemmer = SnowballStemmer("portuguese")

# Aplicar stemmatização nos tokens sem stopwords
stems = " ".join([stemmer.stem(token.text) for token in tokens_no_stop])

print("Stems (primeiros 30):\n\n", stems[:300])


Stems (primeiros 30):

 prolog coraca fanat the imagin pior . mensag diz apen vincent fer . enquant corr ate estaca peg proxim trem ate auvers , pensament vag tras diant . ultim vez hav receb mensag parec , telegram paul gauguin inform vincent gravement enferm . the cheg cidad meridional arles encontr irma enferm hospital 


# Chunking


### Por Tamanho Fixo

In [59]:
# Dividir o texto em chunks utilizando CharacterTextSplitter do LangChain
# corta o texto em pedaços fixos de tamanho, com overlap de aproximadamente 10%
char_splitter = CharacterTextSplitter(
    chunk_size=320,
    chunk_overlap=32,
    separator=".",
)

chunks_char = char_splitter.split_text(lemmas)

print(f"Total de chunks: {len(chunks_char)} \n\n", chunks_char)


Total de chunks: 85 

 ['prologo coracao fanatico theo imaginar mau . mensagem dizer apenas vincent ferir . enquanto correr ate estacao pegar proximo tr ate auver , pensamento vagar tra diante . ultimo vez haver receber mensagem parecido , telegrama paul gauguin informar vincent gravemente enfermo', 'theo chegar cidade meridional arle encontrara irmao enfermaria hospital , cabeca enfaixar perder desvario', 'dessa vez , encontrar final viagem momento assim muito , theo retornar lembranca vincent conhecer passado irmao velho ardoroso irrequieto , tambem cheio brincadeira animar , enorme afinidade infinito capacidade admiracao', 'passeio infancia campo mata redor cidade holandês zundert , onde haver nascir , vincent apresentar beleza misterio natureza . inverno , vincent ensinar patinar andar treno . verao , mostrar construir castelo trilha areia . igreja domingo casa piano sala , cantar voz firme limper', 'quarto sotao dividiar , conversar ate tarde noite , criar irmao novo vinculo irma 

### Recursívo

In [101]:
# Dividir o texto em chunks utilizando RecursiveCharacterTextSplitter do LangChain
# corta o texto, tentando preservar a integridade semântica dos pedaços, utilizando uma lista de separadores.
# os separadores usados nesse caso foram: dupla quebra de linha (para titulos e paragrafos), quebra de linha padrão, ponto e virgula para frases, e espaços para palavras.
# isso visa ir do mais amplo para o mais específico.
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    separators=["\n\n", "\n", ".", ",", " ", ""]
)




chunks_recursive = recursive_splitter.split_text(lemmas)

print(f"Total de chunks: {len(chunks_recursive)}\n\n", chunks_recursive)

Total de chunks: 337

 ['prologo coracao fanatico theo imaginar mau . mensagem dizer apenas vincent ferir', '. enquanto correr ate estacao pegar proximo tr ate auver , pensamento vagar tra diante', '. ultimo vez haver receber mensagem parecido', ', telegrama paul gauguin informar vincent gravemente enfermo', '. theo chegar cidade meridional arle encontrara irmao enfermaria hospital', ', cabeca enfaixar perder desvario', '. dessa vez , encontrar final viagem momento assim muito', ', theo retornar lembranca vincent conhecer passado irmao velho ardoroso irrequieto', ', tambem cheio brincadeira animar , enorme afinidade infinito capacidade admiracao', '. passeio infancia campo mata redor cidade holandês zundert , onde haver nascir', ', vincent apresentar beleza misterio natureza', '. inverno , vincent ensinar patinar andar treno . verao , mostrar construir castelo trilha areia', '. igreja domingo casa piano sala , cantar voz firme limper', '. quarto sotao dividiar , conversar ate tarde noi

# Embedding

## Hugging Face SentenceTransformers

In [110]:
# Transforma os chunks em embeddings, para que possam ser inseridos no banco vetorial.
# Utilizar modelo pré-treinado de Sentence Transformers para gerar embeddings

# outro modelo possivel = "rufimelo/bert-base-portuguese-cased-sts"

# Escolhi o modelo "paraphrase-multilingual-MiniLM-L12-v2" por ser leve e ter boa performance em múltiplos idiomas.
model_name = "paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)

def embed_hf(chunks):
    vectors = model.encode(chunks, show_progress_bar=True)

    print(f"\nDimensão: {vectors.shape}\n\nResultado: \n", vectors[:5])

    return vectors




embedding = embed_hf(chunks_recursive)


Batches: 100%|██████████| 11/11 [00:01<00:00,  8.62it/s]


Dimensão: (337, 384)

Resultado: 
 [[-0.01109407 -0.0033958  -0.09346528 ... -0.07394138  0.06980237
   0.10749811]
 [ 0.23119643  0.1942804  -0.05759234 ...  0.14143626 -0.09300179
   0.23895733]
 [ 0.1060429   0.04165765  0.09783185 ... -0.00279032  0.01778872
  -0.04724207]
 [-0.04810134  0.05285255 -0.16854073 ...  0.09049477  0.03070431
  -0.0620014 ]
 [ 0.08987163 -0.12779948 -0.03018976 ...  0.25491062 -0.11581797
   0.01303199]]


# Banco Vetorial

## Criação/Inserção ChromaDb

In [111]:
# Inicializa o cliente ChromaDB e cria uma coleção para armazenar os embeddings
client = chromadb.Client()
collection_name = "exemplo_colecao"

# resetando coleção caso exista (para testes)
if collection_name in [col.name for col in client.list_collections()]:
    client.delete_collection(collection_name)

collection = client.get_or_create_collection(
    name=collection_name,
    embedding_function=None,
)



print(f"{collection.count()} documentos na coleção")

0 documentos na coleção


In [112]:
# Insere os embeddings na coleção do ChromaDB, criando IDs únicos para cada chunk
ids = [f"chunk_{i}" for i in range(len(chunks_recursive))]

collection.add(
    ids=ids,
    documents=chunks_recursive,
    embeddings=embedding
)




print("Embeddings inseridos com sucesso!")


Embeddings inseridos com sucesso!


## Busca Semântica

In [113]:
# Realiza uma consulta de similaridade no banco vetorial utilizando uma pergunta de exemplo
# Pega uma pergunta, e converte em embeddingm, no mesmo modelo usado para criar os embeddings dos chunks.
question = "Qual o nome de Van Gogh?"
question_embedding = model.encode([question])

# Realiza a consulta no banco vetorial utilizando o embedding da pergunta, e pega os 3 resultados mais similares.
resultados = collection.query(
    query_embeddings=question_embedding.tolist(),
    n_results=3
)

print(f"\nPergunta: {question}")

for doc, score in zip(resultados['documents'][0], resultados['distances'][0]):
    print(f"(Score: {score}) - Documento: {doc}")

# os scores mostram a similaridade entre a pergunta e o chunk retornado. Scores mais baixos indicam maior similaridade.


Pergunta: Qual o nome de Van Gogh?
(Score: 18.112363815307617) - Documento: . Willem , cinco ano idade , avo pintor vincent Willem van gogh
(Score: 19.337644577026367) - Documento: , doenca so reconhecer tarde sobrinho , pintor vincent van gogh
(Score: 21.765853881835938) - Documento: , recorrer Willem carbentus , partir dai passar apresentar encadernador Real
